Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  # division by 4 because of the 2 strides of 2 applied before
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(graph=graph, config=config) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.779306
Minibatch accuracy: 12.5%
Validation accuracy: 12.4%
Minibatch loss at step 100: 1.123852
Minibatch accuracy: 62.5%
Validation accuracy: 67.3%
Minibatch loss at step 200: 0.832292
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.787416
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.479743
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 500: 0.521245
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 600: 0.709345
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
Minibatch loss at step 700: 0.456727
Minibatch accuracy: 87.5%
Validation accuracy: 82.8%
Minibatch loss at step 800: 0.413380
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Minibatch loss at step 900: 0.572103
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 0.640749
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%

Baseline: Test accuracy 90.0%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

def convnet_model(batch_size, num_hidden, beta, keep_training_prob=1, num_steps=3001, lr0=None):
    graph = tf.Graph()
    with graph.as_default():
      # Input data.
      tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
      layer1_biases = tf.Variable(tf.zeros([depth]))
      layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
      layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
      # division by 4 because of the 2 pooling layers
      layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
      layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
      layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
      layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
      dropout_keep_pb = keep_training_prob

      # Model.
      def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + layer1_biases)
        mpool = tf.nn.max_pool(relu,[1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        dropout_out = tf.nn.dropout(mpool, dropout_keep_pb)
        conv = tf.nn.conv2d(dropout_out, layer2_weights, [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + layer2_biases)
        mpool = tf.nn.max_pool(relu,[1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        dropout_out = tf.nn.dropout(mpool, dropout_keep_pb)
        shape = dropout_out.get_shape().as_list()
        reshape = tf.reshape(dropout_out, [shape[0], shape[1] * shape[2] * shape[3]])
        relu_out = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        dropout_out = tf.nn.dropout(relu_out, dropout_keep_pb)
        return tf.matmul(dropout_out, layer4_weights) + layer4_biases

      # Training computation.
      logits = model(tf_train_dataset)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
      loss = loss + beta*tf.nn.l2_loss(layer1_weights) + beta*tf.nn.l2_loss(layer2_weights) + \
            beta*tf.nn.l2_loss(layer3_weights) + beta*tf.nn.l2_loss(layer4_weights)

      
      # Optimizer.
      if lr0 == None:
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
      else:
        global_step = tf.Variable(0)  # count the number of steps taken
        learning_rate = tf.train.exponential_decay(lr0, global_step, 1000, 0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
      test_prediction = tf.nn.softmax(model(tf_test_dataset))


    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(graph=graph, config=config) as session:
      tf.global_variables_initializer().run()
      print('Initialized')
      for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [17]:
convnet_model(batch_size=16, num_hidden=64, beta=1e-04, keep_training_prob=1, num_steps=4001, lr0=0.075)

Initialized
Minibatch loss at step 0: 3.831840
Minibatch accuracy: 12.5%
Validation accuracy: 10.7%
Minibatch loss at step 100: 1.143342
Minibatch accuracy: 68.8%
Validation accuracy: 60.5%
Minibatch loss at step 200: 0.928615
Minibatch accuracy: 81.2%
Validation accuracy: 77.7%
Minibatch loss at step 300: 0.521035
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.559058
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 500: 0.400763
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 600: 0.858966
Minibatch accuracy: 75.0%
Validation accuracy: 83.1%
Minibatch loss at step 700: 0.306835
Minibatch accuracy: 93.8%
Validation accuracy: 84.4%
Minibatch loss at step 800: 0.371139
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Minibatch loss at step 900: 0.457940
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.610743
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### Results

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=1, num_steps=1001): Test acc = 90.1%

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=1, num_steps=2001): Test acc = 92.2%

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=1, num_steps=3001): Test acc = 93.1%

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=1, num_steps=4001): Test acc = 93.6%

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=1, num_steps=5001): Test acc = 93.4%


convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=0.5, num_steps=3001): Test acc = 80.8%

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=0.75, num_steps=3001): Test acc = 90.0%

convnet_model(batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=0.9, num_steps=3001): Test acc = 91.8%

lr_decay?

batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=0.9, num_steps=3001, lr0=0.05; Test acc = 91.8%

batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=0.9, num_steps=3001, lr0=0.1; Test acc = 92%

batch_size=16, num_hidden=64, beta=1e-03, keep_training_prob=1, num_steps=3001, lr0=0.075; Test acc = 92.8%

batch_size=16, num_hidden=64, beta=1e-02, keep_training_prob=1, num_steps=3001, lr0=0.075; Test acc = 91.9%

expnential decay: 0.9

batch_size=16, num_hidden=64, beta=5e-04, keep_training_prob=1, num_steps=3001, lr0=0.075; Test acc = 92.9%

batch_size=16, num_hidden=64, beta=5e-04, keep_training_prob=1, num_steps=3001, lr0=0.05; Test acc = 92.1%

batch_size=16, num_hidden=64, beta=1e-04, keep_training_prob=1, num_steps=3001, lr0=0.05; Test acc = 92.8%

batch_size=16, num_hidden=64, beta=1e-04, keep_training_prob=1, num_steps=3001, lr0=0.075; Test acc = 93.3%

batch_size=16, num_hidden=64, beta=1e-04, keep_training_prob=1, num_steps=4001, lr0=0.075; Test acc = 93.2%

exponential decay: 0.96

batch_size=16, num_hidden=64, beta=1e-04, keep_training_prob=1, num_steps=4001, lr0=0.075; Test acc = 93.7%
